# Augmenting prompts
Using the `{variable}` syntax, we can augment prompts to "%bob". This allows providing necessary information to have code that fits best to requested tasks. This notebook demonstrates how to use this, e.g. to let bob come up with regular expressions for retrieving information from filenames.

In [1]:
filenames = ["exp01_c1_t0001.tif", "exp01_c2_t0001.tif", "exp01_c1_t0002.tif", "exp01_c2_t0002.tif"]

In [2]:
import bia_bob

In [3]:
filenames

['exp01_c1_t0001.tif',
 'exp01_c2_t0001.tif',
 'exp01_c1_t0002.tif',
 'exp01_c2_t0002.tif']

In [5]:
import re

t_list = []
c_list = []

for filename in filenames:
    t_match = re.search(r't(\d+)', filename)
    c_match = re.search(r'c(\d+)', filename)
    
    if t_match:
        t_list.append(int(t_match.group(1)))
    if c_match:
        c_list.append(int(c_match.group(1)))

print("Time points (t_list):", t_list)
print("Channels (c_list):", c_list)

Time points (t_list): [1, 1, 2, 2]
Channels (c_list): [1, 2, 1, 2]


## Computing image object features
The same approach can be used for computing additional features after feature extraction from images.

In [10]:
%%bob
Load blobs.tif. Segmen the bright objects using 
Otsu's method and connected component labeling.
Extract shape descriptors from the label image 
using scikit-image's regionprops_table.
Store the measurements in a pandas DataFrame

In [13]:
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops_table
import pandas as pd

# Load the image
image = imread("blobs.tif")

# Apply Otsu's thresholding
threshold = threshold_otsu(image)
binary = image > threshold

# Perform connected component labeling
label_image = label(binary)

# Extract shape descriptors using regionprops_table
properties = ['label', 'area', 'perimeter', 'centroid', 'bbox', 'eccentricity', 'solidity', 'minor_axis_length', 'major_axis_length']
measurements = regionprops_table(label_image, properties=properties)

# Convert the measurements into a pandas DataFrame
df = pd.DataFrame(measurements)
df.head()

,label,area,perimeter,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,eccentricity,solidity,minor_axis_length,major_axis_length
0,1,433.0,91.254834,13.212471,19.986143,0,10,30,36,0.876649,0.881874,16.819060,34.957399
1,2,185.0,53.556349,4.270270,62.945946,0,53,11,74,0.828189,0.968586,11.803854,21.061417
2,3,658.0,95.698485,12.568389,108.329787,0,95,28,122,0.352060,0.977712,28.278264,30.212552
3,4,434.0,76.870058,9.806452,154.520737,0,144,23,167,0.341084,0.973094,23.064079,24.535398
4,5,477.0,83.798990,13.545073,246.809224,0,237,29,256,0.771328,0.977459,19.833058,31.162612


In [14]:
columns = df.columns
columns

Index(['label', 'area', 'perimeter', 'centroid-0', 'centroid-1', 'bbox-0',
       'bbox-1', 'bbox-2', 'bbox-3', 'eccentricity', 'solidity',
       'minor_axis_length', 'major_axis_length'],
      dtype='object')

In [15]:
%%bob
I have a DataFrame `df` with the following columns: {columns}
How can I put a new column `aspect_ratio` into the table?

In [16]:
# Calculate the aspect ratio
df['aspect_ratio'] = df['major_axis_length'] / df['minor_axis_length']

print(df.columns)

Index(['label', 'area', 'perimeter', 'centroid-0', 'centroid-1', 'bbox-0',
       'bbox-1', 'bbox-2', 'bbox-3', 'eccentricity', 'solidity',
       'minor_axis_length', 'major_axis_length', 'aspect_ratio'],
      dtype='object')
